__Вопросы для повторения__:
* Зачем в С++ нужны шаблоны?
* Сколько здесь будет скомпилировано функций `print`? Какие?

```c++
template<typename T>
void print(const T& x)
{
    std::cout << x << std::endl;
}

int main()
{
    print(5);
    print(6);
    print(5.0);
    print(6.0f);
    print("Dobrynia");
    print("Nikitich");
    print("Alesha");
    print("Popovich");
    print(std::string("Ilya"));
    print(std::string("Muromec"));
}
```

* Сколько раз компилируется шаблон?
* Плюсы и минусы шаблонов?

```c++
template<typename T>
const T& min(const T& a, const T& b)
{
    if (a < b)
        return a;
    else
        return b;
}

int main()
{
    const std::string s = min<std::string>("string 1", "string 2");
    std::cout << s << std::endl;
}
```

* Скомпилируется ли такой код?

```c++
template<typename T>
struct Vector3
{
    T x;
    T y;
    T z;
    
    Vector3<T> operator *(const float value)
    {
        return { x * value, y * value, z * value };
    }
};

std::string f()
{
    Vector3<std::string> v;
    return v.x;
}
```

__Задача__:

Напишите функцию PrintVectorPart, принимающую вектор целых чисел numbers, выполняющую поиск первого отрицательного числа в нём и выводящую в стандартный вывод все числа, расположенные левее найденного, отсортированные в обратном порядке. Если вектор не содержит отрицательных чисел, выведите все числа в обратном порядке без сортировки.

### Немного итераторов

Что такое контейнер и что такое итератор общими словами:
* контейнер - хранилище объектов одного типа
* итератор - "ключик" к конкретному объекту в контейнере, возможно, позволяющий "обходить" контейнер - перечислять объекты в нём

```c++
// контейнер целых чисел
std::vector<int> v = {1, 2, 3, 4, 5};

// итератор, указывающий на нулевой элемент в контейнере:
std::vector<int>::iterator it = v.begin();
// auto it = v.begin();  // или так, компилятор сам выведет тип

++it;  // теперь it указывает на первый элемент в контейнере

*it = 42;

std::cout << v[1]; // # 42
v[1] = 55;
std::cout << *it; // # 55
```

__Вопрос__:
* с какими контейнерами мы уже имели дело в курсе?

<br />

##### Итераторы. begin && end

`begin()`, `end()` - методы у контейнеров, возвращающие итераторы:
* `begin()` на первый элемент в контейнере
* `end()` на следующий за последним элементом

Как, зная `begin()` и `end()`, проверить, что контейнер пуст?

```c++
std::vector<int> v = {10, 20, 30, 40};
```

![](vector_internals.png)

```c++
auto it = v.begin();
std::cout << *it; // 10
        
++it;
std::cout << *it; // 20
        
++it;
std::cout << *it; // 30

++it;
std::cout << *it; // 40

++it; // it == v.end()
std::cout << *it; // UB - you should never dereference end()!
```

<br />

##### range for

https://en.cppreference.com/w/cpp/language/range-for

```c++
std::vector<int> v = {10, 20, 30, 40};
for (int x : v)
    std::cout << x; // во что разворачивается range-for? (упрощённо)
```

во что разворачивается range-for? (упрощённо)

```c++
for (auto x : v) { ... }
```

```c++
{
    auto&& __range = v;
    auto __begin = __range.begin();
    auto __end = __range.end();
    for (; __begin != __end; ++__begin) {
        auto& x = *__begin;
        ...
    }
}
```

Смысл в том, что как только для пользовательского контейнера определены итераторы и методы `begin()`, `end()`, `cbegin()`, `cend()`, для него "из коробки" начинает работать range-for.

<br />

##### Инвалидация итераторов

Итераторы могут быть инвалидированы, если контейнер меняется.

Под инвалидацией итераторов понимается, что они "ломаются молча", о том, что они сломаны, никак не узнать. Программист обязан уметь читать документацию и отслеживать такие ситуации.

Рассмотрим на примере `std::vector` в предположении, что итератор реализован как указатель на элемент в `std::vector`.


```c++
std::vector<int> v = {10, 20, 30, 40};

auto v_end = v.end();
auto it = v.begin();

v.push_back(50);
// at this point:
// |it|    - might be invalidated
// |v_end| - might be invalidated

std::cout << *it; // oooops, ub
if (v.begin() == v_end)  // ooooops, ub
    ...;
```

https://en.cppreference.com/w/cpp/container/vector/push_back

```c++
std::set<int> s = {20, 30, 40, 50};
        
auto it = s.begin();
std::cout << *it;  // 20

s.insert(10);

std::cout << *it;  // ok, 20
```

Почему так? Потому что документация:

https://en.cppreference.com/w/cpp/container/set/insert

*У каждого контейнера у каждого метода прописан контракт на валидность итераторов (когда и какие итераторы инвалидируются).*

*Читайте документацию внимательно!*

<br />

##### Правильное удаление элементов из map/set по условию

Как неправильно удалять элементы из `std::set`:

```c++
std::set<int> s = {1, 2, 3, 4, 5};

auto it = s.begin();
for(; it != s.end(); ++it)
    if((*it) % 2 == 1)
        s.erase(it);
```

В каком месте баг?

Правильное удаление:

```c++
std::set<int> s = {1, 2, 3, 4, 5};
for(auto it = s.begin(); it != s.end();)
{
    if((*it) % 2 == 1)
        it = s.erase(it);
    else
        ++it;
}
```

<br />

Обобщённо проблема выглядит следующим образом:
    
```c++
for (const auto& item : container) {    
    operate_with(item, container);
    // внутри цикла работа с container-ом не должна
    // инвалидировать его итераторы, иначе перебор
    // элементов в контейнере сломается.
}
```

<br />

##### Операции над итераторами доступа

```c++
std::vector<int> v = {10, 20, 30, 40, 50};
        
auto it = v.begin();

// некоторые итераторы позволяют брать следующий элемент:
auto jt_1 = it + 1;
auto jt_2 = std::next(it);
        
// некоторые итераторы позволяют брать предыдущий элемент:
auto jt_3 = it - 1;         // ?
auto jt_4 = std::prev(it);  // ?
        
// некоторые итераторы позволяют прыгать на n элементов вперёд:
auto jt_5 = it + 4;
auto jt_6 = std::advance(it, 4);

// некоторые итераторы позволяют прыгать на n элементов назад:
auto jt_7 = it - 4;                // ?
auto jt_8 = std::advance(it, -4);  // ?

// некоторые итераторы позволяют считать расстояние между ними:
std::cout << std::distance(it, jt_5);  // 4
```

Стандартные операции над итераторами доступа (access iterators):
* `std::next`
* `std::prev`
* `std::advance`
* `std::distance`

<br />

##### Категории итераторов

Как вы помните, у access iterators у `std::forward_list` нельзя делать `--it`, они только для хождения вперёд и только по одному шагу. А у `std::vector` можно вперёд-назад и на любое число шагов за раз. По этому принципу классифицируются итераторы доступа:
* [Forward Iterator](https://en.cppreference.com/w/cpp/named_req/ForwardIterator)
* [Bidirectional Iterator](https://en.cppreference.com/w/cpp/named_req/BidirectionalIterator)
* [Random Access Iterator](https://en.cppreference.com/w/cpp/named_req/RandomAccessIterator)

Классификация имеет важное значение для алгоритмов. Например, алгоритм сортировки работает только с Random Access Iterator:

```c++
std::vector<int> v = {20, 30, 10};
std::sort(v.begin(), v.end());  // ok
        
std::list<int> l = {20, 30, 10};
std::sort(l.begin(), l.end());  // compile-time error
```

И это отражено в требованиях к алгоритму:
    
https://en.cppreference.com/w/cpp/algorithm/sort

Поэтому для `std::list` реализовали свой `sort`:

https://en.cppreference.com/w/cpp/container/list/sort

```c++
std::list<int> l = {20, 30, 10};
l.sort();
```

<br />

Прочие типы итераторов:
* [Input Iterator](https://en.cppreference.com/w/cpp/named_req/InputIterator)
* [Output Iterator](https://en.cppreference.com/w/cpp/named_req/OutputIterator)
* [Reverse Iterator](https://en.cppreference.com/w/cpp/iterator/reverse_iterator)

##### reverse_iterator

![](reverse_iterator.jpg)

```c++
std::vector<int> v = {10, 20, 30, 40};

// сортировка по возрастанию:
std::sort(v.begin(), v.end());
        
// сортировка по убыванию:
std::sort(v.rbegin(), v.rend());
```

Конвертация iterator <-> reverse_iterator:

Обратите внимание на перескакивание итератора на предыдущий элемент при конвертации.

```c++
std::vector<int> v = {10, 20, 30, 40};
        
auto it = v.begin() + 2; // 30

auto rit = std::make_reverse_iterator(it); // 20 !!!!
++rit; // 10

auto it2 = rit.base();  // 20 !!!!
++it2; // 30
```

<br />

#### Специфика итераторов в С++

Формально, итератор - это класс, который обязан определить методы, требуемые его категорией. Других требований нет.

Формально, итератор не обязан итерироваться по контейнерам или итерироваться вообще по чему либо. Итератор может делать любую работу.

Напишем свой собственный итератор по... простым числам:

```c++
class PrimeNumbersIterator {
private:
    std::uint32_t index_ = 0;
    std::uint32_t value_ = 2;
    
public:
    PrimeNumbersIterator(const std::uint32_t index)
        : index_(index)
        , value_(get_nth_prime_number(index))
    {}
    
    PrimeNumbersIterator(const PrimeNumbersIterator&) = default;
    PrimeNumbersIterator(PrimeNumbersIterator&&) = default;
    PrimeNumbersIterator& operator =(const PrimeNumbersIterator&) = default;
    PrimeNumbersIterator& operator = (PrimeNumbersIterator&&) = default;
    
    // ++it
    PrimeNumbersIterator& operator ++() {
        switch_to_next_prime();
        return *this;
    }

    // it++
    PrimeNumbersIterator operator ++(int) {
        PrimeNumbersIterator old = *this;
        switch_to_next_prime();
        return old;
    }
    
    std::uint32_t operator *() const  {
        return value_;
    }
    
    std::uint32_t get_prime_number_index() const {
        return index_;
    }

private:
    void switch_to_next_prime() {
        ++index;
        do {
            ++value_;
        } while (!is_prime(value_));
    }
};

bool operator == (const PrimeNumbersIterator& lhs,
                  const PrimeNumbersIterator& rhs) {
    return
        lhs.get_prime_number_index() ==
        rhs.get_prime_number_index();
}

bool operator != (const PrimeNumbersIterator& lhs,
                  const PrimeNumbersIterator& rhs) {
    return
        lhs.get_prime_number_index() !=
        rhs.get_prime_number_index();
}


// usage:
PrimeNumbersIterator end(5);
for (PrimeNumbersIterator it; it != end; ++it)
    std::cout << *it;
```

<br />

В стандартной библиотеке есть несколько классов итераторов, реализующих "нестандартный" подход:
* `std::istream_iterator<T>` - читает из потока объекты типа T, пока поток не закончится
* `std::ostream_iterator<T>` - выводит в потом объекты типа T через `operator <<`

Мощь и безобразие итераторов в двух примерах:

```c++
std::istringstream str("0.1 0.2 0.3 0.4");
const double sum = std::accumulate(std::istream_iterator<double>(str),
                                   std::istream_iterator<double>(),
                                   0.);


std::vector<int> v = {1, 2, 3, 4, 5};
std::copy(v.begin(),
          v.end(),
          std::ostream_iterator<int>(std::cout, " "));
```

<br />

**Задача**: найти последнее число 5 в последовательности, предшествующее первому 10

Вариант решения:

```c++
template<typename It>
It function(It begin, It end)
{
    auto it = std::find(begin, end, 10);
    
    if (it == end)
        return end;  // no 10
    
    auto rit = std::find(std::make_reverse_iterator(it),
                         std::make_reverse_iterator(begin),
                         5);

    if (rit == std::make_reverse_iterator(begin))
        return end;  // no 5 before 10
    
    return std::next(rit.base());    
}

std::list<int> l = {1, 2, 3, 5, 5, 10};
auto it = function(l.begin(), l.end());
auto rit = function(l.rbegin(), l.rend());
```

<br />

__Задачи__:

	Условие
	Вам требуется написать функцию со следующим заголовком:

std::vector<std::string> Split(const std::string& str, char delimiter);
или
auto Split(const std::string& str, char delimiter) -> std::vector<std::string> 

	Функция должна вернуть вектор строк, полученный разбиением строки str на части по указанному символу-разделителю delimiter.
	Если разделитель встретился в начале или в конце строки str, то в начале (соответственно, в конце) вектора с результатом должна быть пустая строка.
	Если два разделителя встретились рядом, то пустая строка между ними тоже должна попасть в ответ. Для пустой строки надо вернуть пустой вектор.

	Например, Split("What_is_your_name?", '_') должна вернуть вектор из строк What, is, your иname?.
___

	Условие
	Вам требуется написать функцию Join со следующим заголовком:

std::string Join(const std::vector<std::string>& tokens, char delimiter);
или
auto Join(std::vector<std::string>& tokens, char delim) -> std::string;

	Функция должна вернуть строку, полученную склейкой элементов вектора через указанный разделитель.

	Например, Join({"What", "is", "your", "name?"}, '_') должна вернуть строку "What_is_your_name?".

	Примечание
	Если вектор tokens пустой, то функция должна вернуть пустую строку. Если в векторе tokens ровно один элемент, то он и должен вернуться в ответе.
